In [1]:
#Disable Weights & Biases logging
import os
os.environ["WANDB_DISABLED"] = "true"

#Skip reinstalling packages — assume already installed
# !pip install evaluate transformers
# !pip install --upgrade datasets fsspec
!pip install fsspec==2023.6.0

In [2]:
import transformers
print(transformers.__version__)  # Should be 4.53.0
# !pip install --upgrade datasets fsspec
from datasets import load_dataset
from transformers import TrainingArguments, AutoTokenizer, GPT2LMHeadModel, Trainer
import math

4.53.0


In [3]:
# # Clear cache
!rm -rf ~/.cache/huggingface/datasets/wikitext
#Load dataset
dataset = load_dataset("wikitext", name="wikitext-2-v1")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [4]:
# ✅ Tokenization
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  # GPT-2 has no pad token

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [5]:
def tokenize(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=128)

tokenized = dataset.map(tokenize, batched=True, remove_columns=["text"])
block_size = 128

def group_texts(examples):
    concatenated = sum(examples['input_ids'], [])
    total_len = (len(concatenated) // block_size) * block_size
    result = {
        "input_ids": [concatenated[i:i + block_size] for i in range(0, total_len, block_size)],
        "labels": [concatenated[i:i + block_size] for i in range(0, total_len, block_size)]
    }
    return result

Map:   0%|          | 0/4358 [00:00<?, ? examples/s]

Map:   0%|          | 0/36718 [00:00<?, ? examples/s]

Map:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [6]:
lm_dataset = tokenized.map(group_texts, batched=True)

Map:   0%|          | 0/4358 [00:00<?, ? examples/s]

Map:   0%|          | 0/36718 [00:00<?, ? examples/s]

Map:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [7]:
#Load model (only once)
model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Embedding(50257, 768)

In [8]:
#Training config (optimized for speed)
training_args = TrainingArguments(
    output_dir="./gpt2-finetuned",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    logging_strategy="no",             # logging disables
    save_total_limit=1,
    fp16=True,
    eval_strategy="epoch",
    push_to_hub=False
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [9]:
#Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset["train"],
    eval_dataset=lm_dataset["validation"]
)

In [10]:
#Train
trainer.train()

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss
1,No log,1.130038


TrainOutput(global_step=4590, training_loss=1.2109398403458607, metrics={'train_runtime': 855.6175, 'train_samples_per_second': 42.914, 'train_steps_per_second': 5.365, 'total_flos': 2398530207744000.0, 'train_loss': 1.2109398403458607, 'epoch': 1.0})

In [13]:
from google.colab import drive
drive.mount('/content/drive')

# Saving to Google Drive
model.save_pretrained("/content/drive/MyDrive/gpt2-finetuned")
tokenizer.save_pretrained("/content/drive/MyDrive/gpt2-finetuned")


Mounted at /content/drive


('/content/drive/MyDrive/gpt2-finetuned/tokenizer_config.json',
 '/content/drive/MyDrive/gpt2-finetuned/special_tokens_map.json',
 '/content/drive/MyDrive/gpt2-finetuned/vocab.json',
 '/content/drive/MyDrive/gpt2-finetuned/merges.txt',
 '/content/drive/MyDrive/gpt2-finetuned/added_tokens.json',
 '/content/drive/MyDrive/gpt2-finetuned/tokenizer.json')

In [14]:
#Evaluate
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 3.10


In [21]:
from transformers import pipeline

text_generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

prompt = "Worlds biggest fear is blood because"
output = text_generator(prompt, max_length=10, num_return_sequences=1)
print("Generated Text:", output[0]["generated_text"])


Device set to use cuda:0
Both `max_new_tokens` (=256) and `max_length`(=10) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Generated Text: Worlds biggest fear is blood because it is difficult to <unk> the marrow and <unk> the marrow can produce a range of diseases. In the Netherlands, the proportion of patients with <unk> is about 2 %, while in the United States only 0 % of the population is affected. In the United States, there are <unk> of <unk> from <unk> and <unk> <unk> from <unk>. 



In [30]:
import torch
from torch.nn.functional import cross_entropy
from tqdm import tqdm

In [31]:
model.eval()
k = 5  # For Top-5 accuracy
topk_correct = 0
total = 0

In [32]:
val_dataset = lm_dataset["validation"]
subset = val_dataset.select(range(100))  # Using small subset to speed up

In [33]:
for example in tqdm(subset):
    input_ids = torch.tensor([example["input_ids"]]).to(model.device)
    labels = torch.tensor([example["labels"]]).to(model.device)

    with torch.no_grad():
        outputs = model(input_ids)
        logits = outputs.logits  # [1, seq_len, vocab_size]

    # Skip first token, compare prediction at t with true at t+1
    for t in range(logits.shape[1] - 1):
        preds = logits[0, t]
        true_token = labels[0, t + 1]

        topk = torch.topk(preds, k=k).indices  # Top-k token IDs
        if true_token in topk:
            topk_correct += 1
        total += 1
topk_accuracy = topk_correct / total
print(f"Top-{k} Accuracy: {topk_accuracy:.4f}")

100%|██████████| 100/100 [00:04<00:00, 21.32it/s]


In [ ]:
# from transformers import GPT2LMHeadModel, AutoTokenizer

# model = GPT2LMHeadModel.from_pretrained("/content/drive/MyDrive/gpt2-finetuned")
# tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/gpt2-finetuned")
